In [6]:
import typing as T

import numpy as np
import scipy as sp

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

from pydrake.solvers import (  
    MathematicalProgram,
    Solve,
    MathematicalProgramResult
)

import pydrake.geometry.optimization as opt  # pylint: disable=import-error, no-name-in-module
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    HPolyhedron,
    Point
)
from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    Binding,
    QuadraticCost
)
from pydrake.symbolic import Polynomial, Variable, Variables

## Make a simple gcs program

In [46]:

gcs = GraphOfConvexSets()
eps = 0.0001

vs = gcs.AddVertex(Point(np.array([0,0.3])), "s")

v1 = gcs.AddVertex(Point(np.array([0,-2])), "1")
v2 = gcs.AddVertex(Point(np.array([0,2])), "2")

v3 = gcs.AddVertex(HPolyhedron.MakeBox(np.array([2,-2]), np.array([4,2]) ), "v3")

vt = gcs.AddVertex(Point(np.array([5,0])), "t")


es1 = gcs.AddEdge(vs, v1, "s-1")
es2 = gcs.AddEdge(vs, v2, "s-2")

e13 = gcs.AddEdge(v1, v3, "1-3")
e23 = gcs.AddEdge(v2, v3, "2-3")

e3t = gcs.AddEdge(v3, vt, "3-t")


def add_quadratic_cost(edge):
    xu, xv = edge.xu(), edge.xv()
    Q = np.array([[1,0,-1,0],
                  [0,1, 0, -1],
                  [-1,0,1, 0],
                  [0,-1, 0, 1]])
    cost = QuadraticCost(Q = Q, b = np.zeros(4), c = 0)
    edge.AddCost(Binding[QuadraticCost](cost, np.append(xv, xu)))

add_quadratic_cost(es1)
add_quadratic_cost(es2)
add_quadratic_cost(e13)
add_quadratic_cost(e23)
add_quadratic_cost(e3t)



options = GraphOfConvexSetsOptions()
options.convex_relaxation = True
options.max_rounded_paths = 0

solution = gcs.SolveShortestPath(vs, vt, options)

assert solution.is_success()

In [45]:
for e in gcs.Edges():
    print(e.name(), solution.GetSolution(e.phi()))
for v in gcs.Vertices():
    print(v.name(), solution.GetSolution(v.x()))


s-1 0.3730192446244889
s-2 0.6269807553755111
1-3 0.3730192446244889
2-3 0.6269807553755111
3-t 1.0
s [0.  0.3]
1 [ 0.         -1.99999999]
2 [0. 2.]
v3 [2.49999039 0.31218571]
t [5. 0.]


## code for extracting a solution

In [11]:
def find_path_to_target(
    solution: MathematicalProgramResult,
    edges: T.List[GraphOfConvexSets.Edge],
    start: GraphOfConvexSets.Vertex,
    target: GraphOfConvexSets.Vertex,
) -> T.Tuple[T.List[GraphOfConvexSets.Vertex], T.List[GraphOfConvexSets.Edge]]:
    """Given a set of active edges, find a path from start to target. 
    Return a list of vertices in the path and a list of edges in the path.

    Args:
        solution (MathematicalProgramResult): GCS program solution
        edges (T.List[GraphOfConvexSets.Edge]): list of active edges in a GCS solution
        start (GraphOfConvexSets.Vertex): start vertex
        target (GraphOfConvexSets.Vertex): target vertex

    Returns:
        T.Tuple[T.List[GraphOfConvexSets.Vertex], T.List[GraphOfConvexSets.Edge]]: vertex path, edge path
    """
    # seed
    np.random.seed(1)
    # get edges out of the start vertex
    edges_out = [e for e in edges if e.u() == start]
    # get flows out of start vertex
    flows_out = np.array([solution.GetSolution(e.phi()) for e in edges_out])
    proabilities = np.where(flows_out < 0, 0, flows_out) # fix numerical errors
    proabilities /= sum(proabilities) # normalize
    # pick next edge at random
    current_edge = np.random.choice(edges_out, 1, p=proabilities)[0]
    # get the next vertex and continue
    v = current_edge.v()
    # check to see if target has been reached
    target_reached = (v == target)
    # return the list of vertices and edges along the path
    if target_reached:
        return [start] + [v], [current_edge]
    else:
        v, e = find_path_to_target(solution, edges, v, target)
        return [start] + v, [current_edge] + e
    
def get_random_solution_path(
    gcs: GraphOfConvexSets,
    solution: MathematicalProgramResult,
    start: GraphOfConvexSets.Vertex,
    target: GraphOfConvexSets.Vertex
) -> T.Tuple[T.List[GraphOfConvexSets.Vertex], T.List[GraphOfConvexSets.Edge]]:
    """ Extract a path from a solution to a gcs program.

    Args:
        gcs (GraphOfConvexSets): gcs instance
        solution (MathematicalProgramResult): gcs solution from Solve(gcs)
        start (GraphOfConvexSets.Vertex): start vertex
        target (GraphOfConvexSets.Vertex): target vertex

    Returns:
        T.Tuple[T.List[GraphOfConvexSets.Vertex], T.List[GraphOfConvexSets.Edge]]: vertex path, edge path
    """
    flow_variables = [e.phi() for e in gcs.Edges()]
    flow_results = [solution.GetSolution(p) for p in flow_variables]
    active_edges = [edge for edge, flow in zip(gcs.Edges(), flow_results) if flow > 0.0]
    return find_path_to_target(solution, active_edges, start, target)


In [12]:
get_random_solution_path(gcs, solution, vs, vt)

([<pydrake.geometry.optimization.GraphOfConvexSets.Vertex at 0x125dfcdf0>,
  <pydrake.geometry.optimization.GraphOfConvexSets.Edge at 0x12630c7f0>])

In [31]:
prog = MathematicalProgram()
mu_0 = prog.NewContinuousVariables(1)[0]
mu_1 = prog.NewContinuousVariables(1)[0]
mu_2 = prog.NewContinuousVariables(1)[0]
P = np.array([[mu_0, mu_1],[mu_1, mu_2]])
prog.AddPositiveSemidefiniteConstraint(P)
prog.AddLinearConstraint(mu_0-mu_1>=0)
prog.AddLinearConstraint(mu_0+mu_1>=0)
prog.AddLinearConstraint(mu_0-mu_2<=-1)
# prog.AddLinearCost(mu_0+mu_1)

solution = Solve(prog)
print(solution.is_success())
solution.GetSolution(P)

True


array([[ 1.01674224e+00, -1.58616999e-17],
       [-1.58616999e-17,  3.01674224e+00]])

In [32]:
x = np.array([[1,0],[0,2]])
np.linalg.eigvals(x)

array([1., 2.])

In [47]:
np.floor(0.2)

0.0